In [1]:
import os
import random
from glob import glob
from PIL import Image,ImageOps
import matplotlib.pyplot as plt

In [2]:
import cv2
import numpy as np

In [3]:
## creating Dataset
random.seed(1)
IMAGE_SIZE = 128
BATCH_SIZE = 4
MAX_TRAIN_IMAGES = 300

In [4]:
import tensorflow as tf

In [5]:
def autocontrast(tensor, cutoff=0):
    tensor = tf.cast(tensor, dtype=tf.float32)
    min_val = tf.reduce_min(tensor)
    max_val = tf.reduce_max(tensor)
    range_val = max_val - min_val
    adjusted_tensor = tf.clip_by_value(tf.cast(tf.round((tensor - min_val - cutoff) * (255 / (range_val - 2 * cutoff))), tf.uint8), 0, 255)
    return adjusted_tensor

def read_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = autocontrast(image)
    image.set_shape([None, None, 3])
    image = tf.cast(image, dtype=tf.float32) / 255
    return image


def random_crop(low_image, enhanced_image):
    low_image_shape = tf.shape(low_image)[:2]
    low_w = tf.random.uniform(
        shape=(), maxval=low_image_shape[1] - IMAGE_SIZE + 1, dtype=tf.int32
    )
    low_h = tf.random.uniform(
        shape=(), maxval=low_image_shape[0] - IMAGE_SIZE + 1, dtype=tf.int32
    )
    enhanced_w = low_w
    enhanced_h = low_h
    low_image_cropped = low_image[
        low_h : low_h + IMAGE_SIZE, low_w : low_w + IMAGE_SIZE
    ]
    enhanced_image_cropped = enhanced_image[
        enhanced_h : enhanced_h + IMAGE_SIZE, enhanced_w : enhanced_w + IMAGE_SIZE
    ]
    return low_image_cropped, enhanced_image_cropped


def load_data(low_light_image_path, enhanced_image_path):
    low_light_image = read_image(low_light_image_path)
    enhanced_image = read_image(enhanced_image_path)
    low_light_image, enhanced_image = random_crop(low_light_image, enhanced_image)
    return low_light_image, enhanced_image


def get_dataset(low_light_images, enhanced_images):
    dataset = tf.data.Dataset.from_tensor_slices((low_light_images, enhanced_images))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

In [6]:
train_low_light_images = sorted(glob("/content/drive/MyDrive/vlg24/Train/low/*"))[:MAX_TRAIN_IMAGES]
train_enhanced_images = sorted(glob("/content/drive/MyDrive/vlg24/Train/high/*"))[:MAX_TRAIN_IMAGES]

val_low_light_images = sorted(glob("/content/drive/MyDrive/vlg24/Train/low/*"))[MAX_TRAIN_IMAGES:]
val_enhanced_images = sorted(glob("/content/drive/MyDrive/vlg24/Train/high/*"))[MAX_TRAIN_IMAGES:]

In [7]:
train_dataset = get_dataset(train_low_light_images, train_enhanced_images)
val_dataset = get_dataset(val_low_light_images, val_enhanced_images)

In [8]:
import tensorflow as tf

def transformer_network():
    inputs = tf.keras.layers.Input(shape=(None, None, 3))
    batch_size = tf.shape(inputs)[0]

    conv1 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    conv1 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')(conv1)

    conv2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(conv1)
    conv2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(conv2)

    conv3 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')(conv2)
    conv3 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')(conv2)

    res1 = residual_block(conv3, 128)
    res2 = residual_block(res1, 128)
    res3 = residual_block(res2, 128)
    res4 = residual_block(res3, 128)
    res5 = residual_block(res4, 128)

    deconv1 = tf.keras.layers.Conv2DTranspose(64, (3, 3), padding='same', activation='relu')(res5)
    deconv2 = tf.keras.layers.Conv2DTranspose(32, (3, 3), padding='same', activation='relu')(deconv1)

    outputs = tf.keras.layers.Conv2D(3, (3, 3), padding='same', activation='sigmoid')(deconv2)
    outputs=tf.keras.layers.add([inputs, outputs])

    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    return model

def residual_block(inputs, filters):
    x = tf.keras.layers.Conv2D(filters, (3, 3), padding='same', activation='relu')(inputs)
    x = tf.keras.layers.Conv2D(filters, (3, 3), padding='same')(x)
    x = tf.keras.layers.add([inputs, x])
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [9]:
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
for layer in vgg.layers:
    layer.trainable = False
outputs = [vgg.get_layer('block3_conv3').output, vgg.get_layer('block4_conv3').output]
model1 = tf.keras.models.Model(inputs=vgg.input, outputs=outputs)

80134624/80134624 [==============================] - 4s 0us/step


In [10]:
def perceptual_loss(y_true, y_pred):
    y_true_features = model1(y_true)
    y_pred_features = model1(y_pred)
    loss = tf.reduce_mean(tf.square(y_true_features[0] - y_pred_features[0])) + tf.reduce_mean(tf.square(y_true_features[1] - y_pred_features[1]))
    return loss
def charbonnier_loss(y_true, y_pred):
    return tf.reduce_mean(tf.sqrt(tf.square(y_true - y_pred) + tf.square(1e-3)))
def loss(y_true,y_pred):
    return  + 0.5*perceptual_loss(y_true, y_pred)  + 0.4*charbonnier_loss(y_true, y_pred)

In [11]:
def peak_signal_noise_ratio(y_true, y_pred):
    return tf.image.psnr(y_pred, y_true, max_val=1.0)

In [12]:
model = transformer_network()

In [13]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(
    optimizer=optimizer, loss=loss, metrics=[peak_signal_noise_ratio]
)

In [14]:
history = model.fit(train_dataset,validation_data=val_dataset,epochs=50)

Epoch 1/50
75/75 [==============================] - 216s 3s/step - loss: 9.1123 - peak_signal_noise_ratio: 15.0955 - val_loss: 7.1481 - val_peak_signal_noise_ratio: 17.0640
Epoch 2/50
75/75 [==============================] - 18s 233ms/step - loss: 7.3680 - peak_signal_noise_ratio: 16.4283 - val_loss: 6.7372 - val_peak_signal_noise_ratio: 18.0571
Epoch 3/50
75/75 [==============================] - 17s 223ms/step - loss: 6.1180 - peak_signal_noise_ratio: 17.4618 - val_loss: 5.3842 - val_peak_signal_noise_ratio: 19.0046
Epoch 4/50
75/75 [==============================] - 17s 225ms/step - loss: 5.8715 - peak_signal_noise_ratio: 18.1625 - val_loss: 5.4815 - val_peak_signal_noise_ratio: 19.2720
Epoch 5/50
75/75 [==============================] - 16s 206ms/step - loss: 5.3916 - peak_signal_noise_ratio: 18.1750 - val_loss: 5.1786 - val_peak_signal_noise_ratio: 19.2150
Epoch 6/50
75/75 [==============================] - 16s 206ms/step - loss: 5.1000 - peak_signal_noise_ratio: 18.3790 - val_loss

In [15]:
def plot_results(images, titles, figure_size=(12, 12)):
    fig = plt.figure(figsize=figure_size)
    for i in range(len(images)):
        fig.add_subplot(1, len(images), i + 1).set_title(titles[i])
        _ = plt.imshow(images[i])
        plt.axis("off")
    plt.show()


def infer(original_image):
    image = tf.keras.preprocessing.image.img_to_array(original_image)
    image = autocontrast(image)
    image = tf.cast(image,dtype=tf.float32) / 255.0
    image = np.expand_dims(image, axis=0)
    output = model.predict(image,verbose=0)
    output_image = output[0] * 255.0
    output_image = output_image.clip(0, 255)
    output_image = output_image.reshape(
        (np.shape(output_image)[0], np.shape(output_image)[1], 3)
    )
    output_image = Image.fromarray(np.uint8(output_image))
    original_image = Image.fromarray(np.uint8(original_image))
    return output_image

In [23]:
total_psnr = 0.0
num_samples = 0

def calculate_psnr(true_image, generated_image):
    return tf.image.psnr(true_image, generated_image, max_val=1.0)

# Iterate over the dataset
for low_light, normal_light in val_dataset:
    # Generate enhanced images using the generator
    generated_images = model.predict(low_light)

    # Calculate PSNR for each pair in the batch
    for i in range(len(normal_light)):
        psnr_score = calculate_psnr(normal_light[i], generated_images[i])
        total_psnr += psnr_score.numpy()
        num_samples += 1

# Compute the average PSNR score
average_psnr = total_psnr / num_samples
print(f"Average PSNR: {average_psnr:.2f} dB")

1/1 [==============================] - 0s 39ms/step
Average PSNR: 19.63 dB


In [24]:
print(f"Average PSNR: {average_psnr:.2f} dB")

Average PSNR: 19.63 dB
